# Additional Modelling Challenges

## Progressive Party Problem (PPP)

1. Every boat can only hold a limited number of people at a time - the boats' capacities and crew sizes are different.
2. Certain boats are to be designated hosts.
3. The remaining boats visit the host boats for several successive periods.
4. The total number of people aboard a boat, including the host crew and guest crews, must not exceed the capacity.
5. A guest boat cannot revisit a host.
6. Guest crews cannot meet more than once.

Find the visit schedule and host designations that minimises the number of host boats.

In [1]:
import cpmpy as cp
import random

random.seed(0)

# Parameters
n_boats = 10
n_periods = 5

# 1. Every boat can only hold a limited number of people at a time - the boats' capacities and crew sizes are different.
capacity = random.choices(range(10, 150), k=n_boats)
crew_size = [c // 10 for c in capacity]
print("Capacities:", capacity)
print("Crew sizes:", crew_size)

model = cp.Model()

# Decision variables
is_host = cp.boolvar(shape=n_boats)
schedule = cp.intvar(0, n_boats-1, shape=(n_boats, n_periods))

# Constraints
for i in range(n_boats):
    for j in range(n_periods):
        # 2. Certain boats are to be designated hosts.
        model += is_host[i].implies(schedule[i, j] == i)

        # 3. The remaining boats visit the host boats for several successive periods.
        model += [(~is_host[i]).implies(schedule[k, j] != i) for k in range(n_boats)]

        # 4. The total number of people aboard a boat, including the host crew and guest crews, must not exceed the capacity.
        # Assumption: a guest boat's capacity already includes its crew; else its own capacity would be too small for its guests and crew.
        # A host boat on the other hand is assumed to have only a crew, and no guests of its own - since it is a host boat.
        guest_boat_capacities_incl_crews = [(k != i and schedule[k, j] == i) * capacity[k] for k in range(n_boats)]
        model += is_host[i].implies(cp.sum([crew_size[i]] + guest_boat_capacities_incl_crews) <= capacity[i])

    # 5. A guest boat cannot revisit a host.
    model += (~is_host[i]).implies(cp.AllDifferent(schedule[i, :]))

    # 6. Guest crews cannot meet more than once.
    for k in range(n_boats):
        model += ((~is_host[i]) & (~is_host[k]) & (i != k)).implies(cp.sum(schedule[i, :] == schedule[k, :]) <= 1)

# Find the visit schedule and host designations that minimises the number of host boats.
model.minimize(is_host.sum())

if model.solve():
    print("Solution found!")
    print(f"{is_host.value().sum()} hosts: ", [idx for idx, val in enumerate(is_host.value().tolist()) if val is True])
    print("Schedule:\n", schedule.value())
else:
    print("No solution found.")

Capacities: [128, 116, 68, 46, 81, 66, 119, 52, 76, 91]
Crew sizes: [12, 11, 6, 4, 8, 6, 11, 5, 7, 9]
Solution found!
6 hosts:  [0, 1, 4, 6, 8, 9]
Schedule:
 [[0 0 0 0 0]
 [1 1 1 1 1]
 [8 0 1 9 6]
 [4 1 6 0 9]
 [4 4 4 4 4]
 [0 6 8 1 4]
 [6 6 6 6 6]
 [1 4 9 6 8]
 [8 8 8 8 8]
 [9 9 9 9 9]]
